In [26]:
# import sys  
# sys.path.insert(0, '/Users/arturo/Downloads/')

from function2 import *


In [27]:
import pyspark
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.window import Window as W
pd.options.display.max_colwidth = 1000
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
# from pyspark.sql.types import ArrayType, DoubleType, BooleanType
# from pyspark.sql.functions import col,array_contains



In [28]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

# Datos

In [29]:
Nombre_Input_entidades = spark.read.csv("DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP_short1.csv", sep=";", header=True)
Nombre_Input_relaciones = spark.read.csv("DATA_RELACION_ENTIDADES_MAESTRO_short1.csv", sep=";", header=True)
Nombre_Input_investigadores = spark.read.csv("DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO_short1.csv", sep=";", header=True)

# Cruce 1

In [30]:
cruce1 = Nombre_Input_investigadores.\
                                    join(Nombre_Input_relaciones,
                                         Nombre_Input_investigadores.Id ==  Nombre_Input_relaciones.Id_NIVEL_0,"left")\
                                    .dropDuplicates(Nombre_Input_investigadores.columns)\
                                    .filter("Id_NIVEL_0 is not null")

In [31]:
cruce1 = cruce1.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(cruce1.NOMBRE_ENTIDAD_NIVEL_1),
                               cruce1.Centro_Norm )
                          ) 

# DATA_SIC_ENTIDADES_CENTROS_PB

In [32]:
maxId = Nombre_Input_entidades.withColumn("Id", Nombre_Input_entidades.Id.cast('int')).select("Id").rdd.max()[0]

In [33]:
data = cruce1.filter("Similitud <0.875")\
             .withColumn("CIF",lit(None).cast(StringType()))\
             .withColumn("Entidad_Norm",col("Centro_Norm"))\
             .withColumn("Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("ID_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("NIF_COD",lit(None).cast(StringType()))\
             .withColumn("ACRONIMO",lit(None).cast(StringType()))\
             .withColumn("NOMBRE_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("Nombre_Entidad_Mostrar",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N1_1",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N2_1",lit(None).cast(StringType()))\
             .withColumn("DIRECCION_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("COD_CCAA",lit(None).cast(StringType()))\
             .withColumn("CCAA",lit(None).cast(StringType()))\
             .withColumn("ENLACE_WEB",lit(None).cast(StringType()))\
             .withColumn("SOMMA",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_REGIONAL",lit(None).cast(StringType()))\
             .withColumn("ESTADO_x",lit(None).cast(StringType()))\
             .withColumn("CodigoInvente",lit(None).cast(StringType()))\
             .withColumn("DenominacionSocial",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Codigo",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Descripcion",lit(None).cast(StringType()))\
             .withColumn("NIF",lit(None).cast(StringType()))\
             .withColumn("codigoDir3",lit(None).cast(StringType()))\
             .withColumn("codigoOrigen",lit(None).cast(StringType()))\
             .withColumn("Provincia_Codigo",lit(None).cast(StringType()))\
             .withColumn("C_ID_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_ID_NIVEL_ADMON",lit(None).cast(StringType()))\
             .withColumn("C_ID_TIPO_ENT_PUBLICA",lit(None).cast(StringType()))\
             .withColumn("N_NIVEL_JERARQUICO",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("B_SW_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_ESTADO",lit(None).cast(StringType()))\
             .withColumn("D_VIG_ALTA_OFICIAL",lit(None).cast(StringType()))\
             .withColumn("NIF_CIF",lit(None).cast(StringType()))\
             .withColumn("C_ID_AMB_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("C_DESC_PROV",lit(None).cast(StringType()))\
             .withColumn("CONTACTOS",lit(None).cast(StringType()))\
             .withColumn("List_Entidad_Norm",col("Centro_Norm"))\
             .withColumn("List_Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("List_CIF",lit(None).cast(StringType()))

In [34]:
#data = data.dropDuplicates(["Entidad_Norm", "Provincia_Entidad"])
data = data.withColumnRenamed("Id", "IdOld")

In [35]:
Index = [maxId + i for i in range(1, data_count+1)]
Index = spark.createDataFrame(Index, IntegerType())
Index = Index.withColumnRenamed("value", "Id")

In [36]:
windowSpec = W.orderBy("Dummy")

Index = Index.withColumn("Dummy", monotonically_increasing_id())
Index = Index.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.withColumn("Dummy", monotonically_increasing_id())
data = data.withColumn("Dummy", F.row_number().over(windowSpec))

data = data.join(Index, "Dummy", "outer").drop("Dummy")

In [37]:
Nombre_Intermedio_cruce1_entidades = data.select(Nombre_Input_entidades.columns)

In [38]:
# eliminar duplicados

In [39]:
#Nombre_Intermedio_cruce1_entidades.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce1_entidades/csv")

In [40]:
#Nombre_Intermedio_cruce1_entidades.write.format("parquet").mode('overwrite').save("Nombre_Intermedio_cruce1_entidades/parquet")

# Nombre_Output_investigadores

In [41]:
data_2 = cruce1.filter("Similitud >=0.875")\
        .withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id_NIVEL_1"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


In [42]:
data_3 = data.withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(Nombre_Input_investigadores.columns + 
                ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


In [43]:
Nombre_Output_investigadores = data_2.union(data_3)

In [44]:
#result.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Output_investigadores/csv")

In [45]:
#result.write.format("parquet").mode('overwrite').save("Nombre_Output_investigadores/parquet")

# Nombre_Intermedio_cruce1_relaciones

In [46]:
Nombre_Intermedio_cruce1_relaciones = data.select(["Id", "IdOld", "Entidad", "Centro","Centro_Norm", "Provincia_Entidad", "Provincia_Centro","CIF","Entidad_Norm"])

In [47]:
#Eliminamos duplicados

l = get_matching_entities_by_elasticSearch(
                                        Nombre_Input_investigadores, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        Nombre_Input_investigadores, 
                                        "Centro_Norm",
                                        ["Centro_Norm"],
                                        'index_name',
                                        10)

l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l)
duplicados = list(l.filter("final_score >= 0.875").select("Entidad_Norm").dropDuplicates().toPandas().Entidad_Norm)
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.filter(~Nombre_Intermedio_cruce1_relaciones.Entidad_Norm.isin(duplicados))


100%|████████████████████████████████████████| 970/970 [00:04<00:00, 204.97it/s]


22/07/31 11:14:05 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/07/31 11:14:05 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [48]:
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.withColumn("Id_NIVEL_1", col("Id"))\
      .withColumn("Id_NIVEL_0", col("IdOld"))\
      .withColumn("NOMBRE_ENTIDAD_NIVEL_0", col("Entidad"))\
      .withColumn("NOMBRE_ENTIDAD_NIVEL_1", col("Centro"))\
      .withColumn("COD_PROVINCIA_NIVEL_0", col("Provincia_Entidad"))\
      .withColumn("COD_PROVINCIA_NIVEL_1", col("Provincia_Centro"))\
      .withColumn("NIF_COD_NIVEL_0", col("CIF"))\
      .withColumn("Jerarquia", lit("Orgánica Directa").cast(StringType()))\
      .withColumn("ACRONIMO_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ACRONIMO_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("CCAA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("CCAA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("COD_CCAA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("COD_CCAA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("COD_POSTAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("COD_POSTAL_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("DIRECCION_POSTAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("DIRECCION_POSTAL_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ENLACE_WEB_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ESTADO_x_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("ID_ENTIDAD_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("ID_ENTIDAD_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("NIF_COD_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("Nombre_Entidad_Mostrar_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("Nombre_Entidad_Mostrar_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("PROVINCIA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("PROVINCIA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("SOMMA_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("SOMMA_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N1_1_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N1_1_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N2_1_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_N2_1_NIVEL_1",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_REGIONAL_NIVEL_0",lit(None).cast(StringType()))\
      .withColumn("TIPO_ENTIDAD_REGIONAL_NIVEL_1",lit(None).cast(StringType()))\
    

In [49]:
Nombre_Intermedio_cruce1_relaciones = Nombre_Intermedio_cruce1_relaciones.select(Nombre_Input_relaciones.columns)

In [ ]:

#Nombre_Intermedio_cruce1_relaciones.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce1_relaciones/csv")

In [ ]:
#Nombre_Intermedio_cruce1_relaciones.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce1_relaciones/parquet")

# Agregacion vertical de relaciones

In [50]:
Nombre_Output_relaciones = Nombre_Input_relaciones.union(Nombre_Intermedio_cruce1_relaciones)

In [51]:
#Nombre_Intermedio_cruce1_relaciones.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Output_relaciones/csv")

In [52]:
#Nombre_Intermedio_cruce1_relaciones.write.mode('overwrite').format("paequet").save("Nombre_Output_relaciones/parquet")

# If flag_invente_dir3 == True

# Nombre_Input_invente

In [53]:
# data invente
Nombre_Input_invente = pd.read_excel('Analisis_entidades_invente_15-02-2022.xlsx')
for i in Nombre_Input_invente.columns:
    Nombre_Input_invente[i] = Nombre_Input_invente[i].astype(str)



new_header = Nombre_Input_invente.iloc[0] 
# #grab the first row for the header
Nombre_Input_invente = Nombre_Input_invente[1:] #take the data less the header row
Nombre_Input_invente.columns = new_header #set the header row as the df header

Nombre_Input_invente = Nombre_Input_invente[["DenominacionSocial", "Provincia_Codigo", "NIF",
                                                "CodigoInvente", "FormaJuridica_Codigo", 
                                                "FormaJuridica_Descripcion", "codigoDir3","codigoOrigen",
                                                "Provincia_Codigo"]]

Nombre_Input_invente = Nombre_Input_invente.loc[:,~Nombre_Input_invente.columns.duplicated()]

Nombre_Input_invente = spark.createDataFrame(Nombre_Input_invente.fillna('')) 


Nombre_Input_invente = Nombre_Input_invente.withColumn("DenominacionSocial_Norm", UDF_normalizarTexto(Nombre_Input_invente.DenominacionSocial))


# Nombre_Intermedio_cruce2_invente

In [54]:
l = get_matching_entities_by_elasticSearch(
                                        Nombre_Intermedio_cruce1_entidades, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        Nombre_Input_invente, 
                                        "DenominacionSocial_Norm",
                                        Nombre_Input_invente.columns,
                                        'index_name',
                                        1)

22/07/31 11:14:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 11:14:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 11:14:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 11:14:21 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 11:14:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/07/31 11:14:22 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradat

100%|█████████████████████████████████| 112793/112793 [01:21<00:00, 1386.64it/s]


In [55]:
l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l).filter("final_score >= 0.875")

22/07/31 11:15:53 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/07/31 11:15:53 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [56]:
l = l.sort(l.final_score.desc()).dropDuplicates(subset = ['Entidad_Norm']).select("Entidad_Norm","DenominacionSocial", "NIF",
                                                                                "CodigoInvente", "FormaJuridica_Codigo", 
                                                                                "FormaJuridica_Descripcion", "codigoDir3","codigoOrigen",
                                                                                "Provincia_Codigo", "final_score","DenominacionSocial_Norm")


In [57]:
Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce1_entidades.drop("CodigoInvente","DenominacionSocial", "FormaJuridica_Codigo", 
                                                                           "FormaJuridica_Descripcion", "NIF", "codigoDir3", "codigoOrigen",
                                                                            "Provincia_Codigo", "Provincia_Entidad", "CIF")





In [58]:
Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce2_invente.join(l,
                                      Nombre_Intermedio_cruce2_invente.Entidad_Norm ==  l.Entidad_Norm,"left")                                 

     

In [59]:
Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente.filter("final_score is not NULL").drop("Entidad_Norm")
Nombre_Intermedio_cruce2_invente2 = Nombre_Intermedio_cruce2_invente.filter("final_score is  NULL")

In [60]:
Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente1.withColumn("Entidad_Norm", col("DenominacionSocial_Norm"))\
                                                                    .withColumn("Provincia_Entidad", col("Provincia_Codigo"))\
                                                                    .withColumn("CIF", col("NIF"))\
                                                                    .withColumn("Similitud_invente", col("final_score"))

In [61]:
Nombre_Intermedio_cruce2_invente1 = Nombre_Intermedio_cruce2_invente1.select(Nombre_Intermedio_cruce1_entidades.columns+["Similitud_invente"])


In [62]:
Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce2_invente1.union(Nombre_Intermedio_cruce2_invente2)

In [ ]:
#Nombre_Intermedio_cruce2_invente.write.option("header",True).mode('overwrite').format("csv").save("Nombre_Intermedio_cruce2_invente/csv")



In [ ]:
#Nombre_Intermedio_cruce2_invente.write.mode('overwrite').format("paequet").save("Nombre_Intermedio_cruce2_invente/parquet")



# Nombre_Intermedio_proceso_dir3_entidades

In [183]:
Nombre_Input_dir3 = spark.read.csv("DIR3_concatenado_AGE_CCAA_EELL_Universidades.csv", sep=";", header=True).drop("Unnamed: 0")
Nombre_Input_dir3 = Nombre_Input_dir3.withColumn("C_DNM_UD_ORGANICA_Norm", UDF_normalizarTexto(Nombre_Input_dir3.C_DNM_UD_ORGANICA))


In [184]:
l = get_matching_entities_by_elasticSearch(
                                        Nombre_Intermedio_cruce2_invente, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        Nombre_Input_dir3, 
                                        "C_DNM_UD_ORGANICA_Norm",
                                        Nombre_Input_dir3.columns,
                                        'index_name',
                                        10)

l = get_matching_by_elasticSearch_Distance_ratcliff_obershelp(spark,l).filter("final_score >= 0.875")\
                               .dropDuplicates(subset = ['Entidad_Norm', "C_DNM_UD_ORGANICA_Norm"])
                                                                    

100%|█████████████████████████████████| 112793/112793 [01:13<00:00, 1538.42it/s]


22/07/31 14:28:33 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/07/31 14:28:33 WARN StopWordsRemover: Default locale set was [en_MX]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [254]:
unique = l.selectExpr(
              '*', 
              'count(*) over (partition by Entidad_Norm) as cnt'
            ).filter(F.col('cnt') == 1).drop('cnt')



In [255]:
duplicated = l.join(l.groupBy('Entidad_Norm')\
                  .count().where('count = 1').drop('count'),
                on=['Entidad_Norm'],
                how='left_anti')

In [256]:
duplicated1 = duplicated.selectExpr(
              '*', 
              'count(*) over (partition by Entidad_Norm, N_NIVEL_JERARQUICO) as cnt'
            ).filter(F.col('cnt') == 1).drop('cnt')

duplicated1 = duplicated1.sort(duplicated1.N_NIVEL_JERARQUICO.asc())\
                            .dropDuplicates(subset = ['Entidad_Norm'])

In [257]:
duplicated2 = l.join(l.groupBy('Entidad_Norm', 'N_NIVEL_JERARQUICO')\
                  .count().where('count = 1').drop('count'),
                on=['Entidad_Norm','N_NIVEL_JERARQUICO'],
                how='left_anti')


duplicated2 = duplicated2.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(duplicated2.C_DNM_UD_ORGANICA_SUPERIOR),
                               duplicated2.Entidad_Norm )
                          ) 



duplicated2 = duplicated2.sort(duplicated2.Similitud.desc())\
                            .dropDuplicates(subset = ['Entidad_Norm']).drop("Similitud")

In [258]:
match = unique.union(duplicated1).union(duplicated2)\
               .select(['Entidad_Norm',"final_score"] + Nombre_Input_dir3.columns)\
               .withColumnRenamed("Entidad_Norm","Entidad_Norm1")


In [259]:
Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_cruce2_invente\
                                            .drop('C_DNM_UD_ORGANICA_SUPERIOR', 'C_ID_UD_ORGANICA', 'C_ID_TIPO_ENT_PUBLICA',
                                                  'C_DESC_PROV', 'C_ID_DEP_EDP_PRINCIPAL', 'C_ID_ESTADO', 'D_VIG_ALTA_OFICIAL', 
                                                  'CONTACTOS', 'NIF_CIF', 'C_ID_NIVEL_ADMON', 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL', 
                                                  'C_ID_DEP_UD_PRINCIPAL', 'N_NIVEL_JERARQUICO', 'C_DNM_UD_ORGANICA', 
                                                  'C_DNM_UD_ORGANICA_PRINCIPAL', 'B_SW_DEP_EDP_PRINCIPAL', 'C_ID_AMB_PROVINCIA',
                                                  'C_ID_DEP_UD_SUPERIOR')


In [260]:
Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_proceso_dir3_entidades.join(match,
                                      Nombre_Intermedio_proceso_dir3_entidades.Entidad_Norm ==  match.Entidad_Norm1,"left")     



In [261]:
Nombre_Intermedio_proceso_dir3_entidades1 = Nombre_Intermedio_proceso_dir3_entidades.filter("final_score is not NULL").drop("Entidad_Norm")

Nombre_Intermedio_proceso_dir3_entidades2 = Nombre_Intermedio_proceso_dir3_entidades.filter("final_score is  NULL").drop("Entidad_Norm1")

Nombre_Intermedio_proceso_dir3_entidades2 = Nombre_Intermedio_proceso_dir3_entidades2\
                                                .withColumn("Similitud_Dir3",lit(None).cast(FloatType()))

In [262]:
Nombre_Intermedio_proceso_dir3_entidades1 = Nombre_Intermedio_proceso_dir3_entidades1\
                                .withColumn("Entidad_Norm", col("C_DNM_UD_ORGANICA"))\
                                .withColumn("Provincia_Entidad", col("C_DESC_PROV"))\
                                .withColumn("CIF", col("NIF_CIF"))\
                                .withColumn("Similitud_Dir3", col("final_score"))\
                                .drop("Entidad_Norm1")




In [263]:
Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_proceso_dir3_entidades1\
                                            .union(Nombre_Intermedio_proceso_dir3_entidades2)






In [232]:
lista1 = set(Nombre_Intermedio_proceso_dir3_entidades1.columns)

lista2 = set(Nombre_Intermedio_proceso_dir3_entidades2.columns)

difference_1 = set(lista1).difference(set(lista2))

difference_1

set()

In [253]:
k = Nombre_Intermedio_proceso_dir3_entidades2.columns

pd.DataFrame(k)[0].value_counts()

Id                                 1
C_DNM_UD_ORGANICA_SUPERIOR         1
List_Provincia_Entidad             1
List_CIF                           1
Similitud_invente                  1
Entidad_Norm1                      1
final_score                        1
C_ID_UD_ORGANICA                   1
C_DNM_UD_ORGANICA                  1
C_ID_NIVEL_ADMON                   1
C_ID_TIPO_ENT_PUBLICA              1
N_NIVEL_JERARQUICO                 1
C_ID_DEP_UD_SUPERIOR               1
C_ID_DEP_UD_PRINCIPAL              1
Provincia_Codigo                   1
C_DNM_UD_ORGANICA_PRINCIPAL        1
B_SW_DEP_EDP_PRINCIPAL             1
C_ID_DEP_EDP_PRINCIPAL             1
C_DNM_UD_ORGANICA_EDP_PRINCIPAL    1
C_ID_ESTADO                        1
D_VIG_ALTA_OFICIAL                 1
NIF_CIF                            1
C_ID_AMB_PROVINCIA                 1
C_DESC_PROV                        1
CONTACTOS                          1
C_DNM_UD_ORGANICA_Norm             1
List_Entidad_Norm                  1
c

In [ ]:
             \

In [208]:
Nombre_Intermedio_proceso_dir3_entidades1.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = true)
 |-- FormaJuridica_Codigo: string (nul

In [206]:
Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_proceso_dir3_entidades1\
                                                        .union(Nombre_Intermedio_proceso_dir3_entidades2)




AnalysisException: Union can only be performed on tables with the same number of columns, but the first table has 52 columns and the second table has 55 columns;
'Union false, false
:- Project [Id#10956, CIF#14393, Provincia_Entidad#14341, ID_ENTIDAD#7148, NIF_COD#7206, ACRONIMO#7265, NOMBRE_ENTIDAD#7325, Nombre_Entidad_Mostrar#7386, TIPO_ENTIDAD_N1_1#7448, TIPO_ENTIDAD_N2_1#7511, DIRECCION_POSTAL#7575, COD_POSTAL#7640, COD_PROVINCIA#7706, PROVINCIA#7773, COD_CCAA#7841, CCAA#7910, ENLACE_WEB#7980, SOMMA#8051, TIPO_ENTIDAD_REGIONAL#8123, ESTADO_x#8196, CodigoInvente#13918, DenominacionSocial#13915, FormaJuridica_Codigo#13919, FormaJuridica_Descripcion#13920, ... 28 more fields]
:  +- Filter isnotnull(final_score#32306)
:     +- Join LeftOuter, (Entidad_Norm#14290 = Entidad_Norm#32048)
:        :- Project [Id#10956, CIF#14393, Entidad_Norm#14290, Provincia_Entidad#14341, ID_ENTIDAD#7148, NIF_COD#7206, ACRONIMO#7265, NOMBRE_ENTIDAD#7325, Nombre_Entidad_Mostrar#7386, TIPO_ENTIDAD_N1_1#7448

In [194]:
k = Nombre_Intermedio_proceso_dir3_entidades2.columns

In [195]:
pd.DataFrame(k)[0].value_counts()

Entidad_Norm                       2
Id                                 1
C_DNM_UD_ORGANICA_SUPERIOR         1
List_Entidad_Norm                  1
List_Provincia_Entidad             1
List_CIF                           1
Similitud_invente                  1
Similitud_Dir3                     1
C_ID_UD_ORGANICA                   1
C_DNM_UD_ORGANICA                  1
C_ID_NIVEL_ADMON                   1
C_ID_TIPO_ENT_PUBLICA              1
N_NIVEL_JERARQUICO                 1
C_ID_DEP_UD_SUPERIOR               1
C_ID_DEP_UD_PRINCIPAL              1
codigoOrigen                       1
C_DNM_UD_ORGANICA_PRINCIPAL        1
B_SW_DEP_EDP_PRINCIPAL             1
C_ID_DEP_EDP_PRINCIPAL             1
C_DNM_UD_ORGANICA_EDP_PRINCIPAL    1
C_ID_ESTADO                        1
D_VIG_ALTA_OFICIAL                 1
NIF_CIF                            1
C_ID_AMB_PROVINCIA                 1
C_DESC_PROV                        1
CONTACTOS                          1
Provincia_Codigo                   1
c

In [111]:
Nombre_Intermedio_cruce2_invente.drop(str(list3).replace("[", "").replace("]", "")).printSchema()

root
 |-- Id: integer (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = 

"'C_DNM_UD_ORGANICA_SUPERIOR', 'C_ID_UD_ORGANICA', 'C_ID_TIPO_ENT_PUBLICA', 'C_DESC_PROV', 'C_ID_DEP_EDP_PRINCIPAL', 'C_ID_ESTADO', 'Entidad_Norm', 'D_VIG_ALTA_OFICIAL', 'CONTACTOS', 'NIF_CIF', 'C_ID_NIVEL_ADMON', 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL', 'C_ID_DEP_UD_PRINCIPAL', 'N_NIVEL_JERARQUICO', 'C_DNM_UD_ORGANICA', 'C_DNM_UD_ORGANICA_PRINCIPAL', 'B_SW_DEP_EDP_PRINCIPAL', 'C_ID_AMB_PROVINCIA', 'C_ID_DEP_UD_SUPERIOR'"

['C_DNM_UD_ORGANICA_SUPERIOR',
 'C_ID_UD_ORGANICA',
 'C_ID_TIPO_ENT_PUBLICA',
 'C_DESC_PROV',
 'C_ID_DEP_EDP_PRINCIPAL',
 'C_ID_ESTADO',
 'Entidad_Norm',
 'D_VIG_ALTA_OFICIAL',
 'CONTACTOS',
 'NIF_CIF',
 'C_ID_NIVEL_ADMON',
 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL',
 'C_ID_DEP_UD_PRINCIPAL',
 'N_NIVEL_JERARQUICO',
 'C_DNM_UD_ORGANICA',
 'C_DNM_UD_ORGANICA_PRINCIPAL',
 'B_SW_DEP_EDP_PRINCIPAL',
 'C_ID_AMB_PROVINCIA',
 'C_ID_DEP_UD_SUPERIOR']

In [114]:
list1 = Nombre_Intermedio_cruce2_invente.columns
list2 = match.columns

set1 = set(list1)
set2 = set(list2)

set3 = set1 & set2
list3 = list(set3)

list3.remove('Entidad_Norm')
list3

['C_DNM_UD_ORGANICA_SUPERIOR',
 'C_ID_UD_ORGANICA',
 'C_ID_TIPO_ENT_PUBLICA',
 'C_DESC_PROV',
 'C_ID_DEP_EDP_PRINCIPAL',
 'C_ID_ESTADO',
 'D_VIG_ALTA_OFICIAL',
 'CONTACTOS',
 'NIF_CIF',
 'C_ID_NIVEL_ADMON',
 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL',
 'C_ID_DEP_UD_PRINCIPAL',
 'N_NIVEL_JERARQUICO',
 'C_DNM_UD_ORGANICA',
 'C_DNM_UD_ORGANICA_PRINCIPAL',
 'B_SW_DEP_EDP_PRINCIPAL',
 'C_ID_AMB_PROVINCIA',
 'C_ID_DEP_UD_SUPERIOR']

In [104]:
Nombre_Intermedio_cruce2_invente.select('B_SW_DEP_EDP_PRINCIPAL').show()

22/07/31 12:15:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 1

22/07/31 12:15:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 12:15:44 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+---------

['Id',
 'CIF',
 'Entidad_Norm',
 'Provincia_Entidad',
 'ID_ENTIDAD',
 'NIF_COD',
 'ACRONIMO',
 'NOMBRE_ENTIDAD',
 'Nombre_Entidad_Mostrar',
 'TIPO_ENTIDAD_N1_1',
 'TIPO_ENTIDAD_N2_1',
 'DIRECCION_POSTAL',
 'COD_POSTAL',
 'COD_PROVINCIA',
 'PROVINCIA',
 'COD_CCAA',
 'CCAA',
 'ENLACE_WEB',
 'SOMMA',
 'TIPO_ENTIDAD_REGIONAL',
 'ESTADO_x',
 'CodigoInvente',
 'DenominacionSocial',
 'FormaJuridica_Codigo',
 'FormaJuridica_Descripcion',
 'NIF',
 'codigoDir3',
 'codigoOrigen',
 'Provincia_Codigo',
 'C_ID_UD_ORGANICA',
 'C_DNM_UD_ORGANICA',
 'C_ID_NIVEL_ADMON',
 'C_ID_TIPO_ENT_PUBLICA',
 'N_NIVEL_JERARQUICO',
 'C_ID_DEP_UD_SUPERIOR',
 'C_DNM_UD_ORGANICA_SUPERIOR',
 'C_ID_DEP_UD_PRINCIPAL',
 'C_DNM_UD_ORGANICA_PRINCIPAL',
 'B_SW_DEP_EDP_PRINCIPAL',
 'C_ID_DEP_EDP_PRINCIPAL',
 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL',
 'C_ID_ESTADO',
 'D_VIG_ALTA_OFICIAL',
 'NIF_CIF',
 'C_ID_AMB_PROVINCIA',
 'C_DESC_PROV',
 'CONTACTOS',
 'List_Entidad_Norm',
 'List_Provincia_Entidad',
 'List_CIF',
 'Similitud_invente']

In [ ]:
Nombre_Intermedio_cruce2_invente = Nombre_Intermedio_cruce2_invente.join(l,
                                      Nombre_Intermedio_cruce2_invente.Entidad_Norm ==  l.Entidad_Norm,"left")                                 

     

In [98]:
match.printSchema()

root
 |-- Entidad_Norm: string (nullable = true)
 |-- C_ID_UD_ORGANICA: string (nullable = true)
 |-- C_DNM_UD_ORGANICA: string (nullable = true)
 |-- C_ID_NIVEL_ADMON: string (nullable = true)
 |-- C_ID_TIPO_ENT_PUBLICA: string (nullable = true)
 |-- N_NIVEL_JERARQUICO: string (nullable = true)
 |-- C_ID_DEP_UD_SUPERIOR: string (nullable = true)
 |-- C_DNM_UD_ORGANICA_SUPERIOR: string (nullable = true)
 |-- C_ID_DEP_UD_PRINCIPAL: string (nullable = true)
 |-- C_DNM_UD_ORGANICA_PRINCIPAL: string (nullable = true)
 |-- B_SW_DEP_EDP_PRINCIPAL: string (nullable = true)
 |-- C_ID_DEP_EDP_PRINCIPAL: string (nullable = true)
 |-- C_DNM_UD_ORGANICA_EDP_PRINCIPAL: string (nullable = true)
 |-- C_ID_ESTADO: string (nullable = true)
 |-- D_VIG_ALTA_OFICIAL: string (nullable = true)
 |-- NIF_CIF: string (nullable = true)
 |-- C_ID_AMB_PROVINCIA: string (nullable = true)
 |-- C_DESC_PROV: string (nullable = true)
 |-- CONTACTOS: string (nullable = true)
 |-- C_DNM_UD_ORGANICA_Norm: string (nullabl

In [96]:
match.printSchema()

root
 |-- source_name: string (nullable = true)
 |-- target_name: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- C_ID_UD_ORGANICA: string (nullable = true)
 |-- C_DNM_UD_ORGANICA: string (nullable = true)
 |-- C_ID_NIVEL_ADMON: string (nullable = true)
 |-- C_ID_TIPO_ENT_PUBLICA: string (nullable = true)
 |-- N_NIVEL_JERARQUICO: string (nullable = true)
 |-- C_ID_DEP_UD_SUPERIOR: string (nullable = true)
 |-- C_DNM_UD_ORGANICA_SUPERIOR: string (nullable = true)
 |-- C_ID_DEP_UD_PRINCIPAL: string (nullable = true)
 |-- C_DNM_UD_ORGANICA_PRINCIPAL: string (nullable = true)
 |-- B_SW_DEP_EDP_PRINCIPAL: string (nullable = true)
 |-- C_ID_DEP_EDP_PRINCIPAL: string (nullable = true)
 |-- C_DNM_UD_ORGANICA_EDP_PRINCIPAL: string (nullable = true)
 |-- C_ID_ESTADO: string (nullable = true)
 |-- D_VIG_ALTA_OFICIAL: string (nullable = true)
 |-- NIF_CIF: string (nullable = true)
 |-- C_ID_AMB_PROVINCIA: string (nullable = true)
 |-- C_DESC_PROV: string (nullable = true)

In [75]:
duplicated2.sort(duplicated2.Similitud.desc())\
                            .dropDuplicates(subset = ['Entidad_Norm']).toPandas()[["Entidad_Norm","C_DNM_UD_ORGANICA_Norm", "N_NIVEL_JERARQUICO", "C_DNM_UD_ORGANICA_SUPERIOR", "Similitud"]]

,Entidad_Norm,C_DNM_UD_ORGANICA_Norm,N_NIVEL_JERARQUICO,C_DNM_UD_ORGANICA_SUPERIOR,Similitud
0,instituto valenciano de investigaciones agrarias,instituto valenciano de investigaciones agrarias ivia,3,"Conselleria de Agricultura, Desarrollo Rural, Emergencia Climática y Transición Ecológica",0.400000
1,universidad politecnica de cartagena,universidad politecnica de cartagena,1,Universidades,0.489796


In [72]:
duplicated2.toPandas()[["Entidad_Norm","C_DNM_UD_ORGANICA_Norm", "N_NIVEL_JERARQUICO", "C_DNM_UD_ORGANICA_SUPERIOR", "Similitud"]]

,Entidad_Norm,C_DNM_UD_ORGANICA_Norm,N_NIVEL_JERARQUICO,C_DNM_UD_ORGANICA_SUPERIOR,Similitud
0,instituto valenciano de investigaciones agrarias,instituto canario de investigaciones agrarias,3,"Consejería de Agricultura, Ganadería y Pesca",0.373626
1,instituto valenciano de investigaciones agrarias,instituto valenciano de investigaciones agrarias ivia,3,"Conselleria de Agricultura, Desarrollo Rural, Emergencia Climática y Transición Ecológica",0.400000
2,universidad politecnica de cartagena,universidad politecnica de cartagena,1,Universidades,0.489796
3,universidad politecnica de cartagena,universidad politecnica de cataluna,1,Universidades,0.489796


In [69]:
duplicated2.toPandas()[["Entidad_Norm","C_DNM_UD_ORGANICA_Norm", "N_NIVEL_JERARQUICO", "C_DNM_UD_ORGANICA_SUPERIOR"]]

,Entidad_Norm,C_DNM_UD_ORGANICA_Norm,N_NIVEL_JERARQUICO,C_DNM_UD_ORGANICA_SUPERIOR
0,instituto valenciano de investigaciones agrarias,instituto canario de investigaciones agrarias,3,"Consejería de Agricultura, Ganadería y Pesca"
1,instituto valenciano de investigaciones agrarias,instituto valenciano de investigaciones agrarias ivia,3,"Conselleria de Agricultura, Desarrollo Rural, Emergencia Climática y Transición Ecológica"
2,universidad politecnica de cartagena,universidad politecnica de cartagena,1,Universidades
3,universidad politecnica de cartagena,universidad politecnica de cataluna,1,Universidades


In [70]:
duplicated1.toPandas()[["Entidad_Norm","C_DNM_UD_ORGANICA_Norm", "N_NIVEL_JERARQUICO", "C_DNM_UD_ORGANICA_SUPERIOR"]]

,Entidad_Norm,C_DNM_UD_ORGANICA_Norm,N_NIVEL_JERARQUICO,C_DNM_UD_ORGANICA_SUPERIOR
0,universidad de valladolid,universidad de valladolid,1,Universidades


In [ ]:
l = get_matching_entities_by_elasticSearch(
                                        duplicated2, 
                                        "Entidad_Norm", 
                                        ["Entidad_Norm"],
                                        duplicated2, 
                                        "C_DNM_UD_ORGANICA_Norm",
                                        ["N_NIVEL_JERARQUICO"],
                                        'index_name',
                                        10)

In [50]:
l

,source_name,target_name,Entidad_Norm,N_NIVEL_JERARQUICO


In [73]:
duplicated2.toPandas()[["Entidad_Norm","C_DNM_UD_ORGANICA", "N_NIVEL_JERARQUICO"]]

,Entidad_Norm,C_DNM_UD_ORGANICA,N_NIVEL_JERARQUICO
0,universidad politecnica de cartagena,Universidad Politécnica de Cartagena,1
1,universidad politecnica de cartagena,Universidad Politécnica de Cataluña,1
2,instituto valenciano de investigaciones agrarias,Instituto Canario de Investigaciones Agrarias,3
3,instituto valenciano de investigaciones agrarias,Instituto Valenciano de Investigaciones Agrarias (IVIA),3


In [69]:
duplicated1.toPandas()[["Entidad_Norm","C_DNM_UD_ORGANICA", "N_NIVEL_JERARQUICO"]]

,Entidad_Norm,C_DNM_UD_ORGANICA,N_NIVEL_JERARQUICO
0,universidad de valladolid,Universidad de Valladolid,1
1,universidad de valladolid,Museo de la Universidad de Valladolid,2


In [70]:
duplicated.toPandas()[["Entidad_Norm","C_DNM_UD_ORGANICA", "N_NIVEL_JERARQUICO"]]

,Entidad_Norm,C_DNM_UD_ORGANICA,N_NIVEL_JERARQUICO
0,instituto valenciano de investigaciones agrarias,Instituto Canario de Investigaciones Agrarias,3
1,instituto valenciano de investigaciones agrarias,Instituto Valenciano de Investigaciones Agrarias (IVIA),3
2,universidad de valladolid,Museo de la Universidad de Valladolid,2
3,universidad de valladolid,Universidad de Valladolid,1
4,universidad politecnica de cartagena,Universidad Politécnica de Cartagena,1
5,universidad politecnica de cartagena,Universidad Politécnica de Cataluña,1


In [45]:
Nombre_Intermedio_proceso_dir3_entidades = Nombre_Intermedio_cruce2_invente.join(l,
                                      Nombre_Intermedio_cruce2_invente.Entidad_Norm ==  l.Entidad_Norm,"left")                                 


In [47]:
Nombre_Intermedio_proceso_dir3_entidades.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = 

In [44]:
l.select("Entidad_Norm", "C_DNM_UD_ORGANICA_Norm", "N_NIVEL_JERARQUICO","C_DNM_UD_ORGANICA_SUPERIOR")\
                .dropDuplicates(subset = ['Entidad_Norm', "C_DNM_UD_ORGANICA_Norm"]).toPandas()






,Entidad_Norm,C_DNM_UD_ORGANICA_Norm,N_NIVEL_JERARQUICO,C_DNM_UD_ORGANICA_SUPERIOR
0,centro de investigaciones energeticas medioambientales y tecnologicas o a m p,centro de investigaciones energeticas medioambientales y tecnologicas,3,Secretaría General de Investigación
1,consorcio barcelona supercomputing center centro nacional de supercomputacion,consorcio barcelona supercomputing center centro nacional de supercomputacion,5,S.G. de Grandes Instalaciones Cientifico-Tecnicas
2,consorcio centro de investigacion biomedica en red m p,consorcio centro de investigacion biomedica en red,4,Instituto de Salud Carlos III
3,escuela universitaria de relaciones laborales,escuela universitaria de relaciones laborales de zamora,2,Diputación Provincial de Zamora
4,fundacio privada centre de regulacio genomica,fundacio centre de regulacio genomica,2,Ministerio de Ciencia e Innovación
5,fundacion parque cientifico y tecnologico de castilla la mancha,fundacion parque cientifico y tecnologico de castilla la mancha,4,"Secretaria General Educacion, Cultura y Deportes"
6,instituto aragones de ciencias de la salud,instituto aragones de ciencias de la salud,3,Departamento de Sanidad
7,instituto nacional de tecnica aeroespacial esteban terradas,instituto nacional de tecnica aeroespacial esteban terradas inta,3,S. de E. de Defensa
8,instituto valenciano de investigaciones agrarias,instituto canario de investigaciones agrarias,3,"Consejería de Agricultura, Ganadería y Pesca"
9,instituto valenciano de investigaciones agrarias,instituto valenciano de investigaciones agrarias ivia,3,"Conselleria de Agricultura, Desarrollo Rural, Emergencia Climática y Transición Ecológica"


In [43]:
Nombre_Input_dir3.count()

68001

In [ ]:
DIR3_concatenado_AGE_CCAA_EELL_Universidades

In [41]:
Nombre_Intermedio_cruce2_invente.count()

22/07/31 07:18:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 0

22/07/31 07:18:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:28 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 0

22/07/31 07:18:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 07:18:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/31 0

727

In [1]:
from elasticsearch import Elasticsearch
from pyspark.sql.types import FloatType, StructField, StringType, StructType
from tqdm import tqdm
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
import re
import nltk
import textdistance
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql import functions as F
import unidecode

In [8]:
#Nombre_Input_entidades = pd.read_csv("DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", sep=";")
Nombre_Input_relaciones = pd.read_csv("DATA_RELACION_ENTIDADES_MAESTRO.csv", sep=";").head(1000)
Nombre_Input_investigadores = pd.read_csv("DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv", sep=";").head(1000)

In [10]:
def index_es(df,target_names_column,index_name):
    '''
    inputs:
        df : pd.DataFrame object containing the two columns source names and target names
        target_names : type: str, represents the the target names column
        index_name : type: str, represents the index of elastic search object
    output:
        return elasticsearch object indexed, and initialized with target names
    
    '''

    target_names = [str(name).lower() for name in list(df[target_names_column])]
    es = Elasticsearch()
    
    for i in list(es.indices.get_alias("*").items()):
        es.indices.delete(index=list(i)[0])
    
    es.indices.delete(index='test-index', ignore=[400, 404])
    
    
    i=0
    for name in tqdm(target_names):
        doc = {'entity':name}
        es.index(index=index_name, id=i, document=doc)
        i+=1    
    return es

In [62]:
def format_results(results, columnsDf1):
    names_source = []
    names_target = []
    # cities_source=[]
    # cities_target=[]
    # mun_source=[]
    # mun_target=[]
    # countries_source=[]
    # countries_target=[]
    listCol = [[]]*len(columnsDf1)
    for i in results:
        source_name = results[i]['source_name']
        columns = [results[i][name] for name in columnsDf1]
        
        for k in range(len(columnsDf1)):
            listCol[k].append(columns[k])
        
        # source_city = results[i]['source_city']
        # source_mun = results[i]['source_municipality']
        # source_country = results[i]['source_country']
        for element in results[i]['matching_names_by_elasticsearch']:
            target_name = element[1]
            # target_city = element[2]
            # target_mun = element[3]
            # target_country = element[4]
            names_source.append(source_name)
            
            colResult = [col for col in columns]
            # cities_source.append(source_city)
            # mun_source.append(source_mun)
            # countries_source.append(source_country)
            names_target.append(target_name)

                
            # cities_target.append(target_city)
            # mun_target.append(target_mun)
            # countries_target.append(target_country)
            
            
    prev = {'source_name':names_source,'target_name':names_target}        
    
    for l in range(len(columnsDf1)):
        prev[columnsDf1[l]]=listCol[l]
       
            
#     result = pd.DataFrame(prev)
            
    return prev

In [22]:
def format_results(results):
    names_source = []
    names_target = []
    # cities_source=[]
    # cities_target=[]
    # mun_source=[]
    # mun_target=[]
    # countries_source=[]
    # countries_target=[]
    col = []
    for i in results:
        source_name = results[i]['source_name']
        print(results[i])
        test = results[i]['Id_NIVEL_1']
        # source_city = results[i]['source_city']
        # source_mun = results[i]['source_municipality']
        # source_country = results[i]['source_country']
        for element in results[i]['matching_names_by_elasticsearch']:
            target_name = element[1]
            # target_city = element[2]
            # target_mun = element[3]
            # target_country = element[4]
            names_source.append(source_name)
            col.append(test)
            # cities_source.append(source_city)
            # mun_source.append(source_mun)
            # countries_source.append(source_country)
            names_target.append(target_name)
            
            # cities_target.append(target_city)
            # mun_target.append(target_mun)
            # countries_target.append(target_country)
            
            
    prev = {'source_name':names_source,'target_name':names_target
                        }   
    

            
    result = pd.DataFrame()
            
    return result

In [63]:
def get_matching_entities_by_elasticSearch(#syn_cities,
                                           #cities,
                                           df1,
                                           source_names_column,
                                           columnsDf1,
                                        #    city_column_source,
                                        #    municipality_column_source,
                                        #    country_column_source,
                                           df2,
                                           target_names_column,
                                        #    city_column_target,                                          
                                        #    municipality_column_target,                                           
                                        #    country_column_target,
                                           index_name
                                        #    name_weight=0.93283582,city_weight=0.06716418,
                                        #    mun_weight=0.0,country_weight=0.0
                                           ):
    '''
    inputs:
        df : pd.DataFrame object containing the two columns source names and target names
        source_names : type: str, represents the the source names column
        index_name : type: str, represents the index of elastic search object
        size: type: int , number of targets to extract by elasticseasource_pd_altas_nuevas['source_municipality'] = source_pd_altas_nuevas['source_municipality'].astype('Int64')rch, default is 20,
                max for this paremeter is 10 000 
    output:
        return dict object results
    '''
    es = index_es(df2,target_names_column,index_name)

    source_names = list(df1[source_names_column])
    sources=[list(df1[name]) for name in columnsDf1 ]
    

#    cities_target=list(df2[city_column_target])


    # mun_source=list(df1[municipality_column_source])
#    mun_target=list(df2[municipality_column_target])

    # countries_source=list(df1[country_column_source])
#    countries_target=list(df2[country_column_target])
    
    source_idx=0
    results = {}

    for entity_i in tqdm(source_names):

        matching_entities = []
        query = {"from": 0,
                 "size": 10,
                 "query": {"match": {"entity":{'query':entity_i,'fuzziness':'AUTO'}}},
                 "sort":[{'_score':{'order':'desc'}}]}
        resp = es.search(index=index_name, body=query)

        for element in resp['hits']['hits']:

            target_idx = int(element['_id'])
            entity_j = element['_source']['entity']
            matching_entities.append((target_idx,entity_j))            
#            matching_entities.append((target_idx,entity_j,cities_target[target_idx],mun_target[target_idx],countries_target[target_idx]))
            
            Prev = {'source_name':entity_i,'matching_names_by_elasticsearch':matching_entities}

            for i in range(len(columnsDf1)):
                   Prev[columnsDf1[i]]=sources[i]
                
        
  

            results[source_idx] = Prev

        source_idx +=1

    return format_results(results, columnsDf1)
#     results = format_results(results, columnsDf1)

    
#     return results


In [56]:
Nombre_Input_relaciones[Nombre_Input_relaciones["Id_NIVEL_1"]==730]["NOMBRE_ENTIDAD_NIVEL_1"]

53     INSTITUTO MAIMONIDES DE INVESTIGACION BIOMEDIC...
246    INSTITUTO MAIMONIDES DE INVESTIGACION BIOMEDIC...
354    INSTITUTO MAIMONIDES DE INVESTIGACION BIOMEDIC...
Name: NOMBRE_ENTIDAD_NIVEL_1, dtype: object

In [64]:
d = get_matching_entities_by_elasticSearch(
    Nombre_Input_relaciones,
    "NOMBRE_ENTIDAD_NIVEL_1",
    ["Id_NIVEL_1", "Id_NIVEL_0"],
    Nombre_Input_investigadores,
    "Centro_Norm",
    "index_name")

/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/ipykernel_38530/3692108162.py:15: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  for i in list(es.indices.get_alias("*").items()):
  0%|                                                  | 0/1000 [00:00<?, ?it/s]/var/folders/fy/cjds5kvd3692g8__1nlvytj00000gn/T/ipykernel_38530/2887801158.py:53: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.search(index=index_name, body=query)
100%|██████████████████████████████████████| 1000/1000 [00:03<00:00, 281.71it/s]


In [65]:
d.keys()

dict_keys(['source_name', 'target_name', 'Id_NIVEL_1', 'Id_NIVEL_0'])

In [66]:
len(d['source_name'])

5693

In [67]:
len(d['target_name'])

5693

In [68]:
len(d['Id_NIVEL_1'])

1158

In [69]:
len(d['Id_NIVEL_0'])

1158

In [76]:
Nombre_Input_invente.printSchema()

root
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = true)
 |-- FormaJuridica_Codigo: string (nullable = true)
 |-- FormaJuridica_Descripcion: string (nullable = true)
 |-- NIF: string (nullable = true)
 |-- codigoDir3: string (nullable = true)
 |-- codigoOrigen6: string (nullable = true)
 |-- CodigoInvente.1: string (nullable = true)
 |-- EjerceControl: string (nullable = true)
 |-- DenominacionSocial.1: string (nullable = true)
 |-- FechaAlta: string (nullable = true)
 |-- Adscripcion_Codigo: string (nullable = true)
 |-- Adscripcion_Descripcion: string (nullable = true)
 |-- Observaciones: string (nullable = true)
 |-- CodigoOrigen14: string (nullable = true)
 |-- CausaDeAlta_Codigo: string (nullable = true)
 |-- CausaDeAlta_Descripcion: string (nullable = true)
 |-- FormaJuridica_Codigo.1: string (nullable = true)
 |-- FormaJuridica_Descripcion.1: string (nullable = true)
 |-- RegimenContable_Codigo: string (nullable = true)
 |-- RegimenCont

In [88]:
import pandas as pd
dfe = pd.read_csv('DATA_IGAE_INVENTE.csv', on_bad_lines='skip', sep = ',')

In [90]:
dfe.to_csv('DATA_IGAE_INVENTE.csv', sep = ';', index = False)

In [89]:
dfe.columns

Index(['CodigoInvente', 'DenominacionSocial', 'FormaJuridica_Codigo',
       'FormaJuridica_Descripcion', 'NIF', 'codigoDir3', 'codigoOrigen',
       'CodigoInvente.1', 'EjerceControl', 'FechaAlta', 'Adscripcion_Codigo',
       'Adscripcion_Descripcion', 'Observaciones', 'CodigoOrigen',
       'CausaDeAlta_Codigo', 'CausaDeAlta_Descripcion',
       'FormaJuridica_Codigo.1', 'FormaJuridica_Descripcion.1',
       'RegimenContable_Codigo', 'RegimenContable_Descripcion',
       'RegimenControl_Codigo', 'RegimenControl_Descripcion',
       'RegimenPresupuestario_Codigo', 'RegimenPresupuestario_Descripcion',
       'ClasificacionCN_Codigo', 'ClasificacionCN_Descripcion',
       'CondicionMedioPropio', 'ClasificacionEntidad', 'Direccion',
       'Localidad', 'Provincia_Codigo', 'Provincia_Descripcion',
       'CodigoPostal', 'Telefono', 'SitioWeb', 'eMail', 'Denominacion',
       'Codigo', 'PorcentajeParticipacion', 'PorcentajeVoto',
       'Actividad_Situacion_Codigo', 'Actividad_Situacion_D

In [47]:
dfe

,,,,,,,,,CodigoInvente;DenominacionSocial;FormaJuridica_Codigo;FormaJuridica_Descripcion;NIF;codigoDir3;codigoOrigen;CodigoInvente.1;EjerceControl;DenominacionSocial.1;FechaAlta;Adscripcion_Codigo;Adscripcion_Descripcion;Observaciones;CodigoOrigen;CodigoDIR3;CausaDeAlta_Codigo;CausaDeAlta_Descripcion;FormaJuridica_Codigo.1;FormaJuridica_Descripcion.1;RegimenContable_Codigo;RegimenContable_Descripcion;RegimenControl_Codigo;RegimenControl_Descripcion;RegimenPresupuestario_Codigo;RegimenPresupuestario_Descripcion;ClasificacionCN_Codigo;ClasificacionCN_Descripcion;CondicionMedioPropio;ClasificacionEntidad;Direccion;Localidad;Provincia_Codigo;Provincia_Descripcion;CodigoPostal;Telefono;SitioWeb;eMail;Denominacion;Codigo;PorcentajeParticipacion;PorcentajeVoto;Actividad_Situacion_Codigo;Actividad_Situacion_Descripcion;Actividad_Sector_Codigo;Actividad_Sector_Descripcion;ObjetoSocial;Codigo.1;Denominacion.1;Venta_Bienes_DentroSP_Administracion;Venta_Bienes_Dentro_SP_Otras;Venta_Bienes_DentroSP_EntidadesSPI;Venta_Bienes_SectorPrivado;Transf_Subvenciones_Administracion;Transf_Subvenciones_Otras;Transf_Subvenciones_SPI;Transf_Subvenciones_UnionEuropea;Otros_Ingresos;Denominacion.2;FuenteAlta;InformacionFuenteAlta;AltaDesde;FechaVersion;Disponible;FechaBaja
INV00000002;1908 Puerto de Motril,A.I.E.;12;Sociedad mercantil ;V18862987;;0100000XP172;INV00000002;Administración Central;1908 Puerto de Motril,A.I.E.;2008-05-16T00:00:00;0;Varias administraciones públicas;soporte documental ESCRITURA nº 280 de 27 de febrero de 2008 (RM 16/05/2008);;;2;Incorporación al SPI;12;Sociedad mercantil ;2;Privado;3;Auditoría de cuentas;2;Estimativo;5;S.11001 Sociedades no financieras Públicas: Administración Central;;;Recinto Portuario El Varadero,s/n ;Motril ;18;Granada;18613;958601207;https://www.apmotril.com/motrilport ;jmartin@motrilport.com ;Empresa Provincial de Vivienda,Suelo y Equipamiento de Granada,S.A.;INV00005446;29,41;29,41;1;Con actividad económica;2;Empresarial;Gestión y promoción de los intereses comunes de los sectores logístico-portuario y turístico-marítimo del Puerto de Motril,para integrar las sinergias cómo clave de éxito,coordinar la prestación de servicios para mej...
INV00000004;A.I. Abra Industrial,S.A. Sociedad Mercantil Estatal;12;Sociedad mercantil ;A48021349;EA0023191;1097NF;INV00000004;Administración Central;A.I. Abra Industrial,"S.A. Sociedad Mercantil Estatal;1998-07-21T00:00:00;31;Ministerio de Hacienda y Función Pública;""La Sociedad tiene su origen en la constitución con fecha 30 de enero de 1954",de la sociedad española de Minas de Somorrostro,S.A.,procediendo al cambio de denominación social,con fecha 26 de febrero de 1996,a A.I. Abra Industrial,S.A.,NaN
Con fecha 16 de enero de 2017,se inscribe en el Registro la modificación de la denominación social según lo dispuesto en el artículo 111.2 de la Ley 40/2015,de 1 de octubre,de Régimen Jurídico del Sector Público,incluyendo en su denominación “Sociedad Mercantil Estatal”.,NaN,NaN,NaN,NaN,NaN
La actividad principal de la Sociedad se enmarca en el proyecto de promover el desarrollo de actuaciones industriales,tecnológicas y de servicios en los terrenos situados en la zona minera de Bodavalle,recuperando los espacios naturales deteriorados por la acción minera ejercida sobre ellos en años anteriores. En la actualidad no existe la actividad minera,actividad principal por la cual se constituyó la sociedad.,NaN,NaN,NaN,NaN,NaN,NaN
- La urbanización,acondicionamiento,planeamiento,parcelación y reparcelación de terrenos,así como la gestión y la realización de obras y actuaciones precisas para ello.,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
"INV00005465;Zubiaurre-Elkargune Institutua;7;Organismo autónomo;P7001801E;LA0015190;1520017AO002;INV00005465;Ayuntamiento de Azkoitia;Zubiaurre-Elkargune Institutua;1989-03-02T00:00:00;1520017AA000;Ayuntamiento de Azkoitia;soporte documental ESTATUTOS 02.03.1989;1520017AO002;LA0015190;2;Incorporación al SPI;7;Organismo autónom

In [32]:
Nombre_Input_relaciones.columns

['Id_NIVEL_1',
 'Id_NIVEL_0',
 'ID_ENTIDAD_NIVEL_0',
 'ID_ENTIDAD_NIVEL_1',
 'Jerarquia',
 'NIF_COD_NIVEL_0',
 'ACRONIMO_NIVEL_0',
 'NOMBRE_ENTIDAD_NIVEL_0',
 'Nombre_Entidad_Mostrar_NIVEL_0',
 'TIPO_ENTIDAD_N1_1_NIVEL_0',
 'TIPO_ENTIDAD_N2_1_NIVEL_0',
 'DIRECCION_POSTAL_NIVEL_0',
 'COD_POSTAL_NIVEL_0',
 'COD_PROVINCIA_NIVEL_0',
 'PROVINCIA_NIVEL_0',
 'COD_CCAA_NIVEL_0',
 'CCAA_NIVEL_0',
 'ENLACE_WEB_NIVEL_0',
 'SOMMA_NIVEL_0',
 'TIPO_ENTIDAD_REGIONAL_NIVEL_0',
 'ESTADO_x_NIVEL_0',
 'NIF_COD_NIVEL_1',
 'ACRONIMO_NIVEL_1',
 'NOMBRE_ENTIDAD_NIVEL_1',
 'Nombre_Entidad_Mostrar_NIVEL_1',
 'TIPO_ENTIDAD_N1_1_NIVEL_1',
 'TIPO_ENTIDAD_N2_1_NIVEL_1',
 'DIRECCION_POSTAL_NIVEL_1',
 'COD_POSTAL_NIVEL_1',
 'COD_PROVINCIA_NIVEL_1',
 'PROVINCIA_NIVEL_1',
 'COD_CCAA_NIVEL_1',
 'CCAA_NIVEL_1',
 'ENLACE_WEB_NIVEL_1',
 'SOMMA_NIVEL_1',
 'TIPO_ENTIDAD_REGIONAL_NIVEL_1',
 'ESTADO_x_NIVEL_1']

In [15]:
val numbersDf = Seq(
  ("123"),
  ("456"),
  (null),
  ("")
).toDF("numbers")

val lettersDf = Seq(
  ("123", "abc"),
  ("456", "def"),
  (null, "zzz"),
  ("", "hhh")
).toDF("numbers", "letters")

val joinedDf = numbersDf.join(lettersDf, Seq("numbers"))

['Id_NIVEL_1',
 'Id_NIVEL_0',
 'ID_ENTIDAD_NIVEL_0',
 'ID_ENTIDAD_NIVEL_1',
 'Jerarquia',
 'NIF_COD_NIVEL_0',
 'ACRONIMO_NIVEL_0',
 'NOMBRE_ENTIDAD_NIVEL_0',
 'Nombre_Entidad_Mostrar_NIVEL_0',
 'TIPO_ENTIDAD_N1_1_NIVEL_0',
 'TIPO_ENTIDAD_N2_1_NIVEL_0',
 'DIRECCION_POSTAL_NIVEL_0',
 'COD_POSTAL_NIVEL_0',
 'COD_PROVINCIA_NIVEL_0',
 'PROVINCIA_NIVEL_0',
 'COD_CCAA_NIVEL_0',
 'CCAA_NIVEL_0',
 'ENLACE_WEB_NIVEL_0',
 'SOMMA_NIVEL_0',
 'TIPO_ENTIDAD_REGIONAL_NIVEL_0',
 'ESTADO_x_NIVEL_0',
 'NIF_COD_NIVEL_1',
 'ACRONIMO_NIVEL_1',
 'NOMBRE_ENTIDAD_NIVEL_1',
 'Nombre_Entidad_Mostrar_NIVEL_1',
 'TIPO_ENTIDAD_N1_1_NIVEL_1',
 'TIPO_ENTIDAD_N2_1_NIVEL_1',
 'DIRECCION_POSTAL_NIVEL_1',
 'COD_POSTAL_NIVEL_1',
 'COD_PROVINCIA_NIVEL_1',
 'PROVINCIA_NIVEL_1',
 'COD_CCAA_NIVEL_1',
 'CCAA_NIVEL_1',
 'ENLACE_WEB_NIVEL_1',
 'SOMMA_NIVEL_1',
 'TIPO_ENTIDAD_REGIONAL_NIVEL_1',
 'ESTADO_x_NIVEL_1']

In [ ]:
data = data.withColumn("Id", sequence(lit(maxId+1), lit(maxId+data.count())))

In [18]:
a12.withColumn("range", array((1 to 100).map(lit(_)): _*)).show()

SyntaxError: invalid syntax (2611963741.py, line 1)

In [ ]:
data.write.csv("/Users/arturo/repos/repos/test1/data-eng-scripts/test")

In [16]:
data.select("Id").show()

+---+
| Id|
+---+
|833|
|237|
|856|
|930|
|871|
|237|
|849|
|853|
|237|
|237|
|237|
|841|
|847|
|847|
|886|
|861|
|833|
|244|
|859|
|247|
+---+
only showing top 20 rows



In [43]:
numbers_df = spark.parallelize([
    ("123", ), ("456", ), (None, ), ("", )
]).toDF(["numbers"])

letters_df = spark.parallelize([
    ("123", "abc"), ("456", "def"), (None, "zzz"), ("", "hhh")
]).toDF(["numbers", "letters"])

#numbers_df.join(letters_df, numbers_df.numbers.eqNullSafe(letters_df.numbers))

AttributeError: 'SparkSession' object has no attribute 'parallelize'

In [62]:
# list  of employee data
data = [["1", "sravan", "company 1"],
        ["2", "ojaswi", "company 1"],
        ["3", "rohith", "company 2"],
        ["4", "sridevi", "company 1"],
        [None, "bobby", "44"]]
 
# specify column names
columns = ['ID', 'NAME', 'Company']
 
# creating a dataframe from the lists of data
dataframe = spark.createDataFrame(data, columns)
dataframe.show()

+----+-------+---------+
|  ID|   NAME|  Company|
+----+-------+---------+
|   1| sravan|company 1|
|   2| ojaswi|company 1|
|   3| rohith|company 2|
|   4|sridevi|company 1|
|null|  bobby|       44|
+----+-------+---------+



In [63]:
data1 = [["14", "sravan", "company 1"],
        ["2", "ojaswii", "company 1"],
        ["34", "rohith", "company 2"],
        ["4", "sridevi", "company 1"],
        [None, "bobby", None]]
 
# specify column names
columns = ['ID1', 'NAME1', 'Company1']

# creating a dataframe from the lists of data
dataframe1 = spark.createDataFrame(data1, columns)
dataframe1.show()

+----+-------+---------+
| ID1|  NAME1| Company1|
+----+-------+---------+
|  14| sravan|company 1|
|   2|ojaswii|company 1|
|  34| rohith|company 2|
|   4|sridevi|company 1|
|null|  bobby|     null|
+----+-------+---------+



In [64]:
cruce2 = dataframe.join(dataframe1,dataframe.ID.eqNullSafe(dataframe1.ID1))

In [65]:
cruce2.show()

+----+-------+---------+----+-------+---------+
|  ID|   NAME|  Company| ID1|  NAME1| Company1|
+----+-------+---------+----+-------+---------+
|   2| ojaswi|company 1|   2|ojaswii|company 1|
|   4|sridevi|company 1|   4|sridevi|company 1|
|null|  bobby|       44|null|  bobby|     null|
+----+-------+---------+----+-------+---------+



In [61]:
maxid = 1000

In [70]:

mylist = list(range(maxid+ 1,maxid + dataframe.count() + 1))

# notice the parens after the type name
index = spark.createDataFrame(mylist, IntegerType())
index = index.withColumnRenamed("value", "Id")

index.show()

+----+
|  Id|
+----+
|1001|
|1002|
|1003|
|1004|
|1005|
+----+



In [64]:
dataframe = dataframe.withColumn("Dummy", monotonically_increasing_id())
index = index.withColumn("Dummy", monotonically_increasing_id())
df3 = dataframe.join(index, "Dummy", "outer").drop("Dummy")

In [65]:
df3.show()

+---+-------+---------+-----+
| ID|   NAME|  Company|value|
+---+-------+---------+-----+
|  1| sravan|company 1| 1001|
|  2| ojaswi|company 1| 1002|
|  3| rohith|company 2| 1003|
|  4|sridevi|company 1| 1004|
|  5|  bobby|company 1| 1005|
+---+-------+---------+-----+



In [55]:
dataframe.withColumn("Index", col(index.select("value")))

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [52]:
index.show()

+-----+
|value|
+-----+
|   11|
|   12|
|   13|
|   14|
|   15|
+-----+



In [14]:
data_1.select("id").show(data_1.count())

22/07/21 12:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/21 12:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/21 12:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/21 12:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/21 12:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/21 12:36:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/07/21 1

In [ ]:
result = df_1_new.union(df_2_new)

In [32]:
dataframe.show()

+---+-------+---------+--------------------+
| ID|   NAME|  Company|               YEARS|
+---+-------+---------+--------------------+
|  1| sravan|company 1|[11, 12, 13, 14, 15]|
|  2| ojaswi|company 1|[11, 12, 13, 14, 15]|
|  3| rohith|company 2|[11, 12, 13, 14, 15]|
|  4|sridevi|company 1|[11, 12, 13, 14, 15]|
|  5|  bobby|company 1|[11, 12, 13, 14, 15]|
+---+-------+---------+--------------------+



In [26]:
dataframe.withColumn("Idl", udf(lambda id: dates[id])).show()

TypeError: col should be Column

In [ ]:
df = df.repartition(1).withColumn("YEARS",  udf(lambda id: dates[id])

In [24]:
sequence(lit(maxId+1), lit(maxId+dataframe.count()))

Column<'sequence(8199, 8203)'>

In [65]:
fi4.count()

8210

8198

In [55]:
fi4.select("id").show()

+----+
|  id|
+----+
|2668|
|4782|
|4589|
|3515|
|2891|
|7281|
|2635|
| 547|
| 352|
|8193|
|5328|
|6552|
|1491|
|1375|
|6408|
|2344|
|2119|
|8102|
|1528|
|6590|
+----+
only showing top 20 rows



In [48]:
list(set(final.columns) - set(fi4.columns))

[]

In [49]:
len(final.columns)

49

In [50]:
len(fi4.columns)

50

In [45]:
pd.DataFrame(final.columns)[0].value_counts()

C_ID_DEP_UD_PRINCIPAL              2
CIF                                1
codigoDir3                         1
Provincia_Codigo                   1
C_ID_UD_ORGANICA                   1
C_DNM_UD_ORGANICA                  1
C_ID_NIVEL_ADMON                   1
C_ID_TIPO_ENT_PUBLICA              1
N_NIVEL_JERARQUICO                 1
C_ID_DEP_UD_SUPERIOR               1
C_DNM_UD_ORGANICA_SUPERIOR         1
C_DNM_UD_ORGANICA_PRINCIPAL        1
B_SW_DEP_EDP_PRINCIPAL             1
C_ID_DEP_EDP_PRINCIPAL             1
C_DNM_UD_ORGANICA_EDP_PRINCIPAL    1
C_ID_ESTADO                        1
D_VIG_ALTA_OFICIAL                 1
NIF_CIF                            1
C_ID_AMB_PROVINCIA                 1
C_DESC_PROV                        1
CONTACTOS                          1
List_Entidad_Norm                  1
List_Provincia_Entidad             1
codigoOrigen                       1
NIF                                1
Entidad_Norm                       1
FormaJuridica_Descripcion          1
P

In [16]:
fi4.printSchema()

root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [102]:
final.printSchema()

root
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = true)
 |-- FormaJuridica_Codigo: string (nullable = true)
 |-- FormaJu

In [51]:
fi4.printSchema()

root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [6]:
cruce1.columns

['Id',
 'Entidad',
 'Origen_Solicitud',
 'Identificadores_Origen',
 'Entidad_Norm',
 'CIF',
 'CIF_validacion',
 'Centro',
 'Centro_Norm',
 'Tipo',
 'Provincia_Entidad',
 'Pais_Entidad',
 'Provincia_Centro',
 'PIC',
 'Tipo_Persona',
 'Entidad_Match',
 'Provincia_Match',
 'Match',
 'Id_NIVEL_1',
 'Id_NIVEL_0',
 'ID_ENTIDAD_NIVEL_0',
 'ID_ENTIDAD_NIVEL_1',
 'Jerarquia',
 'NIF_COD_NIVEL_0',
 'ACRONIMO_NIVEL_0',
 'NOMBRE_ENTIDAD_NIVEL_0',
 'Nombre_Entidad_Mostrar_NIVEL_0',
 'TIPO_ENTIDAD_N1_1_NIVEL_0',
 'TIPO_ENTIDAD_N2_1_NIVEL_0',
 'DIRECCION_POSTAL_NIVEL_0',
 'COD_POSTAL_NIVEL_0',
 'COD_PROVINCIA_NIVEL_0',
 'PROVINCIA_NIVEL_0',
 'COD_CCAA_NIVEL_0',
 'CCAA_NIVEL_0',
 'ENLACE_WEB_NIVEL_0',
 'SOMMA_NIVEL_0',
 'TIPO_ENTIDAD_REGIONAL_NIVEL_0',
 'ESTADO_x_NIVEL_0',
 'NIF_COD_NIVEL_1',
 'ACRONIMO_NIVEL_1',
 'NOMBRE_ENTIDAD_NIVEL_1',
 'Nombre_Entidad_Mostrar_NIVEL_1',
 'TIPO_ENTIDAD_N1_1_NIVEL_1',
 'TIPO_ENTIDAD_N2_1_NIVEL_1',
 'DIRECCION_POSTAL_NIVEL_1',
 'COD_POSTAL_NIVEL_1',
 'COD_PROVINCIA_NI

In [88]:
setA = set(cruce1.columns)
setB = set(fi4.columns)

print(setA & setB)

{'CIF', 'Entidad_Norm', 'Id', 'Provincia_Entidad'}


['Id',
 'CIF',
 'Entidad_Norm',
 'Provincia_Entidad',
 'ID_ENTIDAD',
 'NIF_COD',
 'ACRONIMO',
 'NOMBRE_ENTIDAD',
 'Nombre_Entidad_Mostrar',
 'TIPO_ENTIDAD_N1_1',
 'TIPO_ENTIDAD_N2_1',
 'DIRECCION_POSTAL',
 'COD_POSTAL',
 'COD_PROVINCIA',
 'PROVINCIA',
 'COD_CCAA',
 'CCAA',
 'ENLACE_WEB',
 'SOMMA',
 'TIPO_ENTIDAD_REGIONAL',
 'ESTADO_x',
 'CodigoInvente',
 'DenominacionSocial',
 'FormaJuridica_Codigo',
 'FormaJuridica_Descripcion',
 'NIF',
 'codigoDir3',
 'codigoOrigen',
 'Provincia_Codigo',
 'C_ID_UD_ORGANICA',
 'C_DNM_UD_ORGANICA',
 'C_ID_NIVEL_ADMON',
 'C_ID_TIPO_ENT_PUBLICA',
 'N_NIVEL_JERARQUICO',
 'C_ID_DEP_UD_SUPERIOR',
 'C_DNM_UD_ORGANICA_SUPERIOR',
 'C_ID_DEP_UD_PRINCIPAL',
 'C_DNM_UD_ORGANICA_PRINCIPAL',
 'B_SW_DEP_EDP_PRINCIPAL',
 'C_ID_DEP_EDP_PRINCIPAL',
 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL',
 'C_ID_ESTADO',
 'D_VIG_ALTA_OFICIAL',
 'NIF_CIF',
 'C_ID_AMB_PROVINCIA',
 'C_DESC_PROV',
 'CONTACTOS',
 'List_Entidad_Norm',
 'List_Provincia_Entidad',
 'List_CIF']

In [14]:
cruce1.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Entidad: string (nullable = true)
 |-- Origen_Solicitud: string (nullable = true)
 |-- Identificadores_Origen: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- CIF_validacion: string (nullable = true)
 |-- Centro: string (nullable = true)
 |-- Centro_Norm: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- Pais_Entidad: string (nullable = true)
 |-- Provincia_Centro: string (nullable = true)
 |-- PIC: string (nullable = true)
 |-- Tipo_Persona: string (nullable = true)
 |-- Entidad_Match: string (nullable = true)
 |-- Provincia_Match: string (nullable = true)
 |-- Match: string (nullable = true)
 |-- Id_NIVEL_1: string (nullable = true)
 |-- Id_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_1: string (nullable = true)
 |-- Jerarquia: string (nullable = true)
 |--

In [12]:
fi4.printSchema()


root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [21]:
a12.printSchema()

root
 |-- Id_NIVEL_1: string (nullable = true)
 |-- Id_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_1: string (nullable = true)
 |-- Jerarquia: string (nullable = true)
 |-- NIF_COD_NIVEL_0: string (nullable = true)
 |-- ACRONIMO_NIVEL_0: string (nullable = true)
 |-- NOMBRE_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- Nombre_Entidad_Mostrar_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1_NIVEL_0: string (nullable = true)
 |-- DIRECCION_POSTAL_NIVEL_0: string (nullable = true)
 |-- COD_POSTAL_NIVEL_0: string (nullable = true)
 |-- COD_PROVINCIA_NIVEL_0: string (nullable = true)
 |-- PROVINCIA_NIVEL_0: string (nullable = true)
 |-- COD_CCAA_NIVEL_0: string (nullable = true)
 |-- CCAA_NIVEL_0: string (nullable = true)
 |-- ENLACE_WEB_NIVEL_0: string (nullable = true)
 |-- SOMMA_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL_NIVEL_0: string (nullab